In [3]:
from numba import cuda

def is_gpu_available():
    try:
        return cuda.is_available()
    except:
        return False

gpu_available = is_gpu_available()
print(f"GPU is available: {gpu_available}")

GPU is available: False


In [4]:
if gpu_available:
    import cudf as pd
    from cuml.preprocessing import LabelEncoder, StandardScaler
    from cuml.model_selection import train_test_split
    from cuml.metrics import accuracy_score
else:
    import pandas as pd
    from sklearn.preprocessing import LabelEncoder, StandardScaler
    from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Read Data
use cudf to read data into GPU

- delete unuseful colums(id, Name, City)
- Combine[(Academic Pressure, Work Pressure), (Working professional or student, Profession), (Study Satisfaction, Job Satisfaction)]
- Normalize or mapping

In [5]:
df = pd.read_csv("/content/train.csv")


# drop 'id' , 'Name' (Not revelant), 'city'
df.drop(['id', 'Name'], axis=1, inplace=True)
df.drop(['City'], axis=1, inplace=True)
# combine 'working professional or student' and 'Profession'
df['Working Professional or Student'] = df['Profession'].fillna(df['Working Professional or Student'])
df.drop(['Profession'], axis=1, inplace=True)
# combine 'Academic Pressure' and 'Work Pressure'
df['Academic Pressure'] = df['Work Pressure'].fillna(df['Academic Pressure'])
df.drop(['Work Pressure'], axis=1, inplace=True)
# combine 'Study Satisfaction' and 'Job Satisfaction'
df['Study Satisfaction'] = df['Job Satisfaction'].fillna(df['Study Satisfaction'])
df.drop(['Job Satisfaction'], axis=1, inplace=True)
# most of 'CGPA' leave null, drop it
df.drop(['CGPA'], axis=1, inplace=True)

# one - hot encoder
df['Working Professional or Student'] = LabelEncoder().fit_transform(df['Working Professional or Student'])
df['Degree'] = LabelEncoder().fit_transform(df['Degree'])
df['Academic Pressure'] = LabelEncoder().fit_transform(df['Academic Pressure'])


# bool
df['Have you ever had suicidal thoughts ?'] = df['Have you ever had suicidal thoughts ?'].map({'Yes': 1, 'No': 0})
df['Family History of Mental Illness'] = df['Family History of Mental Illness'].map({'Yes': 1, 'No': 0})
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})
df['Sleep Duration'] = df['Sleep Duration'].map({'Less than 5 hours': 0, '5-6 hours': 1, '7-8 hours': 2, 'More than 8 hours': 3})
df['Dietary Habits'] = df['Dietary Habits'].map({'Healthy': 0, 'Moderate': 1, 'Unhealthy': 2})

# normalize all constant value
cols_to_scale = [
    'Age', 'Academic Pressure', 'Study Satisfaction',
    'Work/Study Hours', 'Financial Stress'
]
scaler = StandardScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])


df.head(10)

,Gender,Age,Working Professional or Student,Academic Pressure,Study Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,0.695360,10,1.406358,-0.689253,3.0,0.0,33,0,-1.363057,-0.699607,0,0
1,1,-1.161867,55,0.693248,0.022383,0.0,2.0,63,1,0.193928,0.007793,0,1
2,1,-0.596624,54,1.406358,-0.689253,1.0,0.0,21,1,-0.844062,-1.407006,0,1
3,1,-1.484863,55,1.406358,-1.400889,0.0,1.0,28,1,0.972421,-1.407006,1,1
4,0,-0.838871,9,-1.446083,-1.400889,1.0,2.0,28,1,0.712923,0.715193,1,0
5,0,1.502850,26,-0.732973,1.445655,1.0,0.0,82,0,0.193928,1.422593,0,0
6,1,0.533862,11,1.406358,-0.689253,2.0,1.0,83,0,-0.065570,-0.699607,0,0
7,1,-0.192879,55,-0.019863,0.734019,2.0,2.0,21,0,0.972421,0.007793,1,0
8,0,-1.323365,54,-0.732973,1.445655,1.0,1.0,36,0,-0.844062,-0.699607,1,0
9,0,0.130117,22,0.693248,-1.400889,1.0,0.0,84,1,0.193928,-0.699607,1,0


In [6]:
# delete any row with null value
df = df.dropna()

X = df.drop(columns=['Depression'])
y = df['Depression']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [7]:
def evaluate_model(y_true, y_pred):
    if hasattr(y_true, 'to_array'):
        y_true = y_true.to_array()
    if hasattr(y_pred, 'to_array'):
        y_pred = y_pred.to_array()

    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("Classification Report:\n", classification_report(y_true, y_pred))


In [9]:
if gpu_available:
    from cuml.linear_model import LogisticRegression
    from cuml.ensemble  import RandomForestClassifier
    from cuml.svm import SVC
    from cuml.naive_bayes import GaussianNB
    from cuml.neighbors import KNeighborsClassifier
else:
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.svm import SVC
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier

# Random Forest
model = RandomForestClassifier(criterion="log_loss", class_weight='balanced',
                               n_estimators=120, random_state=102)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("-" * 50 + "Random Forest" + "-" * 50)
evaluate_model(y_test, y_pred)

--------------------------------------------------Random Forest--------------------------------------------------
Accuracy: 0.9357638271385381
Confusion Matrix:
 [[22241   766]
 [ 1040  4068]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.97      0.96     23007
           1       0.84      0.80      0.82      5108

    accuracy                           0.94     28115
   macro avg       0.90      0.88      0.89     28115
weighted avg       0.93      0.94      0.94     28115



# Machine Learning Method

In [6]:
if gpu_available:
    from cuml.linear_model import LogisticRegression
    from cuml.ensemble  import RandomForestClassifier
    from cuml.svm import SVC
    from cuml.naive_bayes import GaussianNB
    from cuml.neighbors import KNeighborsClassifier
else:
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.svm import SVC
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier

# Logistic Regression
model = LogisticRegression(penalty="elasticnet", max_iter=500)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("-" * 50 + "Losgistic Regression" + "-" * 50)
evaluate_model(y_test, y_pred)


# Random Forest
model = RandomForestClassifier(criterion="log_loss", class_weight='balanced',
                               n_estimators=120, random_state=102)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("-" * 50 + "Random Forest" + "-" * 50)
evaluate_model(y_test, y_pred)


# SVM
model = SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("-" * 50 + "SVM" + "-" * 50)
evaluate_model(y_test, y_pred)

# To-do: still debugging
# # Naive Bayes
# model = GaussianNB()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# print("-" * 50 + "Naive Bayes" + "-" * 50)
# evaluate_model(y_test, y_pred)

# KNN
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("-" * 50 + "KNN" + "-" * 50)
evaluate_model(y_test, y_pred)



--------------------------------------------------Losgistic Regression--------------------------------------------------
Accuracy: 0.9298239373999644
Confusion Matrix:
 [[22143   864]
 [ 1109  3999]]
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.96      0.96     23007
           1       0.82      0.78      0.80      5108

    accuracy                           0.93     28115
   macro avg       0.89      0.87      0.88     28115
weighted avg       0.93      0.93      0.93     28115

--------------------------------------------------Random Forest--------------------------------------------------
Accuracy: 0.9373643962297706
Confusion Matrix:
 [[22248   759]
 [ 1002  4106]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.97      0.96     23007
           1       0.84      0.80      0.82      5108

    accuracy                           0.94     28115
   macro avg  

In [1]:
#XGBoost
from xgboost import XGBClassifier

tree_method = 'gpu_hist' if gpu_available else 'hist'
predictor = 'gpu_predictor' if gpu_available else 'cpu_predictor

model = XGBClassifier(tree_method=tree_method, predictor=predictor,
                      use_label_encoder=False, eval_metrix="logloss")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("-" * 50 + "XGBoost" + "-" * 50)
evaluate_model(y_test, y_pred)

SyntaxError: unterminated string literal (detected at line 5) (<ipython-input-1-5990139100f5>, line 5)

# Neural Network